# Cluster description

In [11]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

env = dotenv_values(".env")
openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
len(df)
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


Apply data model

In [2]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster
Cluster 0    86
Cluster 1    53
Cluster 3    46
Cluster 2    44
Name: count, dtype: int64

Get cluster descriptions based on OpenAI

In [15]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

print(cluster_descriptions['Cluster 0'])

age - 35-44: 35, 45-54: 26, 55-64: 10, 25-34: 8, >=65: 3, unknown: 2, 18-24: 1, <18: 1
edu_level - Wyższe: 85, Podstawowe: 1, Średnie: 0
fav_animals - Psy: 50, Inne: 12, Koty: 11, Brak ulubionych: 9, Koty i Psy: 4
fav_place - Nad wodą: 61, Inne: 0, W górach: 0, W lesie: 0
gender - Mężczyzna: 61, Kobieta: 24



In [16]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 0:
age - 35-44: 35, 45-54: 26, 55-64: 10, 25-34: 8, >=65: 3, unknown: 2, 18-24: 1, <18: 1
edu_level - Wyższe: 85, Podstawowe: 1, Średnie: 0
fav_animals - Psy: 50, Inne: 12, Koty: 11, Brak ulubionych: 9, Koty i Psy: 4
fav_place - Nad wodą: 61, Inne: 0, W górach: 0, W lesie: 0
gender - Mężczyzna: 61, Kobieta: 24


Klaster Cluster 2:
age - 45-54: 13, 35-44: 12, 25-34: 10, 18-24: 8, >=65: 1, 55-64: 0, <18: 0, unknown: 0
edu_level - Średnie: 44, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 20, Koty: 12, Inne: 8, Brak ulubionych: 4, Koty i Psy: 0
fav_place - W górach: 18, Nad wodą: 12, W lesie: 7, Inne: 3
gender - Mężczyzna: 37, Kobieta: 7


Klaster Cluster 1:
age - 45-54: 21, 35-44: 17, 25-34: 12, 55-64: 3, 18-24: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 53, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 53, Brak ulubionych: 0, Inne: 0, Koty: 0, Koty i Psy: 0
fav_place - W górach: 27, W lesie: 23, Inne: 3, Nad wodą: 0
gender - Mężczyz

In [18]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [19]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [20]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [21]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Miłośnicy Wody i Psów', 'description': 'W tym klastrze znajdują się głównie mężczyźni w wieku 35-54 lat z wyższym wykształceniem, którzy preferują spędzanie czasu nad wodą i mają szczególne upodobanie do psów.'}, 'Cluster 1': {'name': 'Górscy Entuzjaści Psów', 'description': 'W tym klastrze dominują mężczyźni w wieku 35-54 lat z wyższym wykształceniem, którzy preferują spędzanie czasu w górach i są zdecydowanymi miłośnikami psów.'}, 'Cluster 2': {'name': 'Młodzi Miłośnicy Gór i Zwierząt', 'description': 'W tym klastrze znajdują się głównie młodsi mężczyźni w wieku 18-54 lat ze średnim wykształceniem, którzy preferują spędzanie czasu w górach i mają różnorodne upodobania zwierzęce, z przewagą psów i kotów.'}, 'Cluster 3': {'name': 'Kociarze Górscy', 'description': 'W tym klastrze dominują mężczyźni w wieku 35-54 lat z wyższym wykształceniem, którzy preferują spędzanie czasu w górach i mają szczególne upodobanie do kotów, choć część z nich nie ma ulubionych zwierz

In [22]:
df_with_clusters[df_with_clusters['Cluster'] == 'Cluster 2']

,age,edu_level,fav_animals,fav_place,gender,Cluster
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 2
3,35-44,Średnie,Koty,W górach,Mężczyzna,Cluster 2
9,18-24,Średnie,Koty,W górach,Mężczyzna,Cluster 2
13,35-44,Średnie,Inne,W górach,Mężczyzna,Cluster 2
15,35-44,Średnie,Koty,Nad wodą,Mężczyzna,Cluster 2
24,45-54,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 2
31,35-44,Średnie,Psy,Nad wodą,Kobieta,Cluster 2
33,25-34,Średnie,Inne,Inne,Mężczyzna,Cluster 2
34,45-54,Średnie,Koty,Inne,Mężczyzna,Cluster 2
35,25-34,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 2
